In [ ]:
import os
from libv3.utils import *
import pandas as pd
import json

In [ ]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, _ = get_config('mamba2_config')   ### config stored in libv3/exp_config.txt
VER = 3
base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

In [ ]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

In [ ]:
paths_traces[0]    ## 14, 17

In [ ]:
## checking if all keys are constant
# for i in range(len(varlist_path)):
varlist1 = read_json(varlist_path[0])
varlist2 = read_json(varlist_path[1])
# varlist3 = read_json(varlist_path[2])
# varlist4 = read_json(varlist_path[3])
# print(varlist.keys())
keys = list(varlist1.keys())
keys.sort()
print(keys)
# print(len(varlist1.keys()), len(varlist2.keys()), len(varlist3.keys()), len(varlist4.keys()))
print(len(varlist1.keys()), len(varlist2.keys()))

# for k in keys:
#     print(varlist1[k], varlist2[k], varlist3[k], varlist4[k])
#     # print(k)
#     if not (varlist1[k] == varlist2[k] == varlist3[k] == varlist4[k]): 
#         print(k)


In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    check_con, _ = is_consistent(varlist_path)

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        print('varlist is not consistent')


In [ ]:
to_number

In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

In [ ]:
sorted_keys

## Trace Plots

### Process Traces

In [ ]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

### Generate plot trace data

In [ ]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [ ]:
all_df[2]

In [ ]:

### plot single trace
for i, df in enumerate(all_df):
    
    if i == trace_file:
        plot_single_trace(df, sorted_keys, with_time=False, is_xticks=False)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


In [ ]:
len(to_plot[6][-1])
to_plot[6][-1]

### Generate execution interval plots

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [ ]:
### count and prepare labels to plot
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
class_count = defaultdict(int)
for i, path in enumerate(paths_label):
    label_content = prepare_gt(path)
    ind, ts, cls = label_content
    # print(ind, ts, cls)
    for c in cls:
        class_count[c]+=1
        
    if i == trace_file:
        print(path)
        toplot_gt = label_content

    print(os.path.split(path)[-1], class_count)

    # break
for key, val in class_count.items():
    print(key, val)

In [ ]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_obj = plot_single_trace(df, var_list, with_time=False, is_xticks=False, ground_truths=toplot_gt)
        plot_obj.show()

## Generate train data

In [ ]:
# '''
# Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
# The format is as follows:
# path_traces = [path1, path2, ...]
# normal_seq_inter = [ ( intervals for trace1, ...), 
#                     ( intervals for trace2 ...),
#                       ...] 
# '''

# normal_seq_inter = (((0,6200), (6500,9910)),
#                     ((0,8260), (8260,11000)))  ### v3, mamba2 trial1, trial2
            

# for p, n_inter in zip(paths_traces, normal_seq_inter):
#     trace = read_traces(p)
#     train_data_path = os.path.join(os.path.dirname(p), 'train_data')

#     if not os.path.exists(train_data_path):
#         os.makedirs(train_data_path)

#     for i, inter in enumerate(n_inter):
#         start, end = inter
#         # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
#         json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
paths_traces

## label correction

In [ ]:
### read the  label and trace
### the timestamps in labels are not found in the trace

for trace_read, labels_read in zip(paths_traces, paths_label):
    print(trace_read, labels_read)
    labels_write = labels_read.replace('labels/', 'labels_corrected_ts/')
    print(labels_write)

    trace_data = read_json(trace_read)
    label_data = read_json(labels_read)

    print(trace_data)
    timestamp = [x[1] for x in trace_data]
    # print(timestamp)
    # print(label_data)
    labels_dict = label_data['labels'].keys()
    print(labels_dict)
    for key in labels_dict:
        print(key)
        labels = label_data['labels'][key]  ### only single key
        new_labels = []
        for l in labels:
            # print(l)
            ts1 = timestamp[l[0]]
            ts2 = timestamp[l[1]]
            new_labels.append((l[0], l[1], ts1, ts2, l[4]))
            # print((l[0], l[1], ts1, ts2, l[4]))

            # break
        # print(labels)
        # print(new_labels)
        print(label_data['labels'][key])
        label_data['labels'][key] = new_labels
        print(label_data['labels'][key])

    ### write the corrected labels
    if not os.path.exists(os.path.dirname(labels_write)):
        os.makedirs(os.path.dirname(labels_write))

    if not os.path.exists(labels_write):
        json.dump(label_data, open(labels_write, 'w'))
    else:
        raise ValueError('file already exists')
    # break

## Convert to V4

In [ ]:
### get paths

v4_paths_traces = [x.replace('version_3', 'version_4') for x in paths_traces if '.DS_Store' not in x]
v4_varlist_path = [x.replace('version_3', 'version_4') for x in varlist_path if '.DS_Store' not in x]

In [ ]:
print(v4_paths_traces)
print(v4_varlist_path)

### Traces

In [ ]:
import shutil

### read traces and remove duplicates
removed_indices = []
for path_read, path_write in zip(paths_traces, v4_paths_traces):
    # print(path_read, path_write)
    trace = read_traces(path_read)
    # print(trace)
    new_trace = []
    for i, t in enumerate(trace):
        event = t[0]
        timestamp = t[1]

        if i == 0:
            last_event = event
            new_trace.append(t)
        else:
            if event == last_event:
                removed_indices.append((i, timestamp))
                # print('Duplicate event:', event, timestamp)
            else:
                new_trace.append(t)
                last_event = event
            # print(t)
    print(np.array(trace).shape)
    print(np.array(new_trace).shape)
    print(trace)
    print(new_trace)

    #### save the new trace to V4
    if not os.path.exists(os.path.dirname(path_write)):
        os.makedirs(os.path.dirname(path_write))

    if os.path.exists(path_write):
        raise Exception('File already exists:', path_write)
    else:
        json.dump(new_trace, open(path_write, 'w'))
        print('Saved:', path_write)
    # break

### copy varlist
for path_read, path_write in zip(varlist_path, v4_varlist_path):
    print(path_read, path_write)
    shutil.copy(path_read, path_write)
    print('Copied:', path_write)

### Labels

In [ ]:
#### translate the labels for faulty data

v4__paths_label = [x.replace('version_3', 'version_4') for x in paths_label if '.DS_Store' not in x]

for trace_read, labels_read, labels_write in zip(v4_paths_traces, paths_label, v4__paths_label):
    print(trace_read)
    print(labels_read, labels_write)

    label_data = read_json(labels_read)

    trace_data = read_traces(trace_read)
    # print(trace_data)
    # timestamp = [x[1] for x in trace_data]
    # print(timestamp)

    ### update label data
    # keys = label_data['metadata'].keys()
    # print(keys)
    print(label_data['metadata'])
    label_data['metadata']['version'] = 4
    label_data['metadata']['path'] = trace_read
    print(label_data['metadata'])

    labels = label_data['labels']
    keys = label_data['labels'].keys()
    # print(keys)
    # print(labels)

    new_labels_all = []
    for l in keys:
        old_labels = labels[l]
        for label in old_labels:
            new_label = np.array((0,0,0,0,0))
            ind1, ind2, ts1, ts2, cls = label
            # print(ind1, ind2, ts1, ts2, cls)
            prev_ts = 0
            found_ts1 = False
            found_ts2 = False
            for i, (event, timestamp) in enumerate(trace_data):
                # print(timestamp)
                if found_ts1 == False:
                    diff_ts =  timestamp - ts1
                    # print(diff_ts)
                    if diff_ts >= 0:
                        print('ts1:', ind1, ts1)
                        print('prev:', prev_ts)
                        print('current:', i, timestamp, diff_ts)
                        found_ts1 = True
                        ### get most suitable index
                        if abs(prev_ts[2]) < abs(diff_ts):
                            new_label[0] = prev_ts[0]
                            new_label[2] = prev_ts[1]
                        else:
                            new_label[0] = i
                            new_label[2] = timestamp
                            break     
                # elif found_ts1 == True and found_ts2 == False:
                #     diff_ts =  timestamp - ts2
                #     print(diff_ts)
                #     if diff_ts >= 0:
                #         print('ts2:', ind2, ts2)
                #         print('prev:', prev_ts)
                #         print('current:', i, timestamp, diff_ts)
                #         found_ts2 = True
                #         ### get most suitable index
                #         if abs(prev_ts[2]) < abs(diff_ts):
                #             new_label[1] = prev_ts[0]
                #             new_label[3] = prev_ts[1]
                #         else:
                #             new_label[1] = i
                #             new_label[3] = timestamp
                #         break
                        
                prev_ts = (i, timestamp, diff_ts)
            new_label[1] = new_label[0] + (ind2-ind1)
            # print(new_label[1], len(trace_data))
            # print(trace_data[new_label[1]])
            new_label[3] = trace_data[new_label[1]][1]
            new_label[4] = cls
            new_label = new_label.tolist()
            # print(new_label)
            new_labels_all.append(new_label)
        print('old_labels:', old_labels)
        print('new_labels', new_labels_all)
        print(label_data)
        label_data['labels'][l] = new_labels_all
        print(label_data)
        # break

    ### save the new labels
    if not os.path.exists(os.path.dirname(labels_write)):
        os.makedirs(os.path.dirname(labels_write))
    
    if os.path.exists(labels_write):
        raise Exception('File already exists:', labels_write)
    else:
        json.dump(label_data, open(labels_write, 'w'))

    # break
    